In [87]:
import requests  
import math  
import pandas as pd  
import time
import urllib
import urllib.request
from bs4 import BeautifulSoup as Bs
import random
random.seed(36)

def get_json(url,num,pos):  
    '''''从网页获取JSON,使用POST请求,加上头部信息'''  
    my_headers = {  
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36',  
            'Host':'www.lagou.com',  
            'Referer':'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput=',  
            'X-Anit-Forge-Code':'0',  
            'X-Anit-Forge-Token': 'None',  
            'X-Requested-With':'XMLHttpRequest'  
            }  

    my_data = {  
            'first': 'true',  
            'pn':num,  
            'kd':pos}  

    res = requests.post(url, headers = my_headers, data = my_data)  
    res.raise_for_status()  
    res.encoding = 'utf-8'  
    # 得到包含职位信息的字典  
    page = res.json()  
    return page


def get_page_num(count):  
    '''''计算要抓取的页数'''  
    # 每页15个职位,向上取整  
    res = math.ceil(count/15)  
    # 拉勾网最多显示30页结果  
    if res > 30:  
        return 30  
    else:  
        return res  

def get_page_info(jobs_list):  
    '''''对一个网页的职位信息进行解析,返回列表'''  
    page_info_list = []  
    for n, i in enumerate(jobs_list): 
        print('现在进行到第{}个；'.format(n))
        job_info = []  
        job_info.append(i['companyFullName'])  
        job_info.append(i['companyShortName'])  
        job_info.append(i['companySize'])  
        job_info.append(i['financeStage'])  
        job_info.append(i['district'])  
        job_info.append(i['positionName'])  
        job_info.append(i['workYear'])  
        job_info.append(i['education'])  
        job_info.append(i['salary'])  
        job_info.append(i['positionAdvantage'])  
        job_info.append(i['positionId'])
        
        url = 'https://www.lagou.com/jobs/'+ str(i['positionId']) +'.html'
        headers = {  
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36',  
                'Host':'www.lagou.com',  
                'Referer':'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput=',  
                'X-Anit-Forge-Code':'0',  
                'X-Anit-Forge-Token': 'None',  
                'X-Requested-With':'XMLHttpRequest'  
                }  

        #data = urllib.parse.urlencode(dict).encode('utf-8')
        #data参数如果要传必须传bytes（字节流）类型的，如果是一个字典，先用urllib.parse.urlencode()编码。
        time_sleep = random.randint(10, 30)
        time.sleep(time_sleep)  
        request = urllib.request.Request(url = url, headers = headers,method = 'POST')
        response = urllib.request.urlopen(request)
        html = response.read().decode('utf-8')
        soup = Bs(html, 'html.parser')
        try:
            content = soup.find('dd', class_='job_bt').get_text()
            job_info.append(content)
        except:
            print(i['positionId'])
        finally:    
            page_info_list.append(job_info)  
    return page_info_list  

In [ ]:

random.seed(36)

url = 'https://www.lagou.com/jobs/positionAjax.json?px=new&needAddtionalResult=false'
#url = 'https://www.lagou.com/jobs/positionAjax.json?city=%E6%B7%B1%E5%9C%B3&needAddtionalResult=false'  
# 先设定页数为1,获取总的职位数  
pos = input('请输入你想查阅的职位名称：')
page_1 = get_json(url,1,pos)  
total_count = page_1['content']['positionResult']['totalCount']  
num = get_page_num(total_count)  
total_info = []  
time_sleep = random.randint(10, 30)
time.sleep(time_sleep)  
print('职位总数:{},页数:{}'.format(total_count,num))  

for n in range(1,num+1):  
    # 对每个网页读取JSON, 获取每页数据  
    page = get_json(url,n,pos)  
    jobs_list = page['content']['positionResult']['result']  
    page_info = get_page_info(jobs_list)  
    total_info += page_info  
    print('已经抓取第{}页, 职位总数:{}'.format(n, len(total_info)))  
    # 每次抓取完成后,暂停一会,防止被服务器拉黑  
    time_sleep = random.randint(10, 30)
    time.sleep(time_sleep)  
#将总数据转化为data frame再输出  
df = pd.DataFrame(data = total_info,columns = ['公司全名','公司简称','公司规模','融资阶段','区域','职位名称','工作经验','学历要求','工资','职位福利','职位描述'])   
df.to_csv('lagou_jobs.csv',index = False)  
print('已保存为csv文件.')  

请输入你想查阅的职位名称：量化
职位总数:2621,页数:30
现在进行到第0个；
现在进行到第1个；
现在进行到第2个；


In [84]:
total_info

[['深圳市三只小熊科技有限公司',
  '三只小熊',
  '150-500人',
  '不需要融资',
  '南山区',
  '交易员',
  '1-3年',
  '本科',
  '15k-25k',
  '.,.,.,.',
  5086316,
  '\n职位描述：\n\n寻找优秀交易员，从事高频程序化交易。【岗位职责】1、负责公司管理的十余个交易所高频做市与套利账户的盘中实时盯盘。2、负责盘中特殊情况的人工干预与处理。3、负责结合市场实际情况的交易策略开发与改进。4、负责各账户的风险管理与统计清算工作。【任职条件】1、立志成为职业操盘手\xa02、有期货公司、基金公司、证券公司工作或实习经验者\xa03、有商品期货/股指期货实盘交易经验者或外汇/数字货币实盘交易经验者优先\xa04、有程序化交易经验者优先\xa05、愿意从事夜盘交易工作者\xa06、极强的执行力与纪律性\xa0\n\n'],
 ['富途网络科技（深圳）有限公司',
  '富途网络',
  '500-2000人',
  'C轮',
  '南山区',
  '交易及结算系统工程师',
  '不限',
  '本科',
  '15k-30k',
  '互联网金融,高可用系统,分布式系统,直连交易所',
  4194141,
  '\n职位描述：\n\n职位诱惑：1、能够熟悉券商的内部运作机制、风控模型，并进行创新；2、公司快速成长中，大量新业务、系统改造进行中，锻炼机会多，上升空间大；3、学习气氛好，有各种培训课程，有内部知识分享平台，技能在深度和广度上都能得到提高；4、能学习到大量金融知识，接触到很多炒股高手，有助于投资理财能力的成长。5、运动气氛好，公司有健身房、冲凉房，有不少运动达人。工作职责：1、负责交易及结算系统的日常开发和维护工作；2、参与系统改造、重点项目、技术难点的攻坚。工作要求：1、熟练掌握C/C++开发，熟悉网络编程，具有扎实的数据结构和算法功底；2、熟悉linux开发环境，熟悉MySQL的使用方法；3、具备良好的分析解决问题能力、学习能力、团队协作能力，认真负责；4、熟练掌握Python者优，有金融产品研发经验者优。\n\n本职位长期招聘，也欢迎应届毕业生投递简历。\n\n\n'],
 ['深圳比特投资咨询有限公司',
  '深圳比特投资',
  '少于15人',
  '未融资

In [71]:
content[n:m+n]

'要求：'